<a href="https://colab.research.google.com/github/Pramo92/Deep-Learning-Mini-Project-03/blob/main/English_to_Sinhala_Translation_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#English to Sinhala Translation with Transformers

##Necessary Library Imports


In [22]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

##Prepare the Data


###Mount the Google Drive

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Read the data file


In [28]:
text_file = "/content/drive/My Drive/dataset1"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break


Go.	යන්න.
Hi.	ආයුබෝවන්.
Run.	දුවන්න.
Who?	කවුද?
Wow!	වාව්!
Fire!	ගිනි!
Help!	උදව්!
Hide.   සඟවන්න.	
Jump.   පනින්න.	
Stop!   නවත්වන්න!	
Wait!   ඉන්න!	
Wait.   ඉන්න.	
Begin.  ආරම්භය.	
Go on.  යන්න.		
Hello.	ආයුබෝවන්.
I ran.	මම දිව්වා.
I try.	මම උත්සාහ කරනවා.
I won!	මම දිනුවා!
Oh no!	අපොයි නෑ!
Relax.	සන්සුන් වන්න.


In [25]:
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

Death is something that we're often discouraged to talk about or even think about, but I've realized that preparing for death is one of the most empowering things you can do. Thinking about death clarifies your life.    මරණය යනු අප බොහෝ විට කතා කිරීමට හෝ ඒ ගැන සිතීමට අධෛර්යමත් කරන දෙයකි, නමුත් මරණයට සූදානම් වීම ඔබට කළ හැකි වඩාත්ම බලගතු දෙයක් බව මම තේරුම් ගතිමි. මරණය ගැන සිතීම ඔබේ ජීවිතය පැහැදිලි කරයි.	
Since there are usually multiple websites on any given topic, I usually just click the back button when I arrive on any webpage that has pop-up advertising. I just go to the next page found by Google and hope for something less irritating.  ඕනෑම මාතෘකාවක් මත සාමාන්‍යයෙන් වෙබ් අඩවි කිහිපයක් ඇති බැවින්, මම සාමාන්‍යයෙන් උත්පතන වෙළඳ දැන්වීම් ඇති ඕනෑම වෙබ් පිටුවකට පැමිණෙන විට ආපසු බොත්තම ක්ලික් කරන්න. මම Google විසින් සොයා ගන්නා ලද ඊළඟ පිටුවට ගොස් අඩු කුපිත කරවන දෙයක් බලාපොරොත්තු වෙමි.	
If someone who doesn't know your background says that you sound like a native speaker, it means they probab

###Split the English and Spanish translation *pairs*

In [29]:
text_pairs = []
for line in lines:
    # Check if the line contains exactly one tab character
    if line.count("\t") == 1:
        english, sinhala = line.split("\t", 1)  # Split only once
        sinhala = "[start] " + sinhala.strip() + " [end]"  # strip to remove leading/trailing whitespace
        text_pairs.append((english.strip(), sinhala))  # strip to remove leading/trailing whitespace

for i in range(3):
    print(random.choice(text_pairs))


('He told his men they would attack the next night.', '[start] ඔහු තම මිනිසුන්ට පැවසුවේ ඔවුන් ඊළඟ රාත්\u200dරියේ පහර දෙන බවයි. [end]')
('A thick hedge surrounded the garden.', '[start] වත්ත වටේ ඝන වැටක්. [end]')
('I have a friend whose father is a magician.', '[start] මගේ යාළුවෙක් ඉන්නවා එයාගේ තාත්තා මැජික් කාරයෙක්. [end]')


###Randomize the data

In [30]:
import random
random.shuffle(text_pairs)

###Spliting the data into training, validation and Testing

In [33]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))
print("Total size of the dataset:",len(train_pairs)+len(val_pairs)+len(test_pairs))

Total sentences: 17992
Training set size: 12596
Validation set size: 2698
Testing set size: 2698
Total size of the dataset: 17992


###Removing Punctuation

In [34]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [35]:
f"{3+5}"

'8'

###Vectorizing the English and Spanish text pairs

In [36]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]


In [37]:
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_sinhala_texts)

###Preparing datasets for the translation task

In [38]:
batch_size = 64
def format_dataset(eng, sin):
    eng = source_vectorization(eng)
    sin = target_vectorization(sin)
    return ({
        "english": eng,
        "sinhala": sin[:, :-1],
    }, sin[:, 1:])
def make_dataset(pairs):
    eng_texts, sin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    sin_texts = list(sin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")
inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
print(list(train_ds.as_numpy_iterator())[50])

inputs['english'].shape: (52, 20)
inputs['sinhala'].shape: (52, 20)
targets.shape: (52, 20)
({'english': array([[  33,   80,   51, ...,    0,    0,    0],
       [  69,    8,  707, ...,    0,    0,    0],
       [   4,   94,   42, ...,    0,    0,    0],
       ...,
       [   4,  165, 1448, ...,    0,    0,    0],
       [ 130,   30,    3, ...,    0,    0,    0],
       [   5,  450,  239, ...,    0,    0,    0]]), 'sinhala': array([[   2,   19, 3151, ...,    0,    0,    0],
       [   2, 1856, 4779, ...,    0,    0,    0],
       [   2,    6, 1269, ...,    0,    0,    0],
       ...,
       [   2,    4,   81, ...,    0,    0,    0],
       [   2,    4,   64, ...,    0,    0,    0],
       [   2,    5,   11, ...,    0,    0,    0]])}, array([[  19, 3151, 2781, ...,    0,    0,    0],
       [1856, 4779,   49, ...,    0,    0,    0],
       [   6, 1269,   50, ...,    0,    0,    0],
       ...,
       [   4,   81,  313, ...,    0,    0,    0],
       [   4,   64, 4024, ...,    0,    0, 

###Transformer encoder implemented as a subclassed Layer

In [39]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

###The Transformer decoder

In [40]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


###Positional Encoding


In [41]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

###End-to-end Transformer

In [42]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [43]:
transformer.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding_2 (Po  (None, None, 256)            3845120   ['english[0][0]']             
 sitionalEmbedding)                                                                               
                                                                                                  
 positional_embedding_3 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']       

###Training the sequence-to-sequence Transformer

In [44]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=28, validation_data=val_ds)

Epoch 1/28
197/197 [==============================] - 612s 3s/step - loss: 5.8207 - accuracy: 0.2963 - val_loss: 5.1850 - val_accuracy: 0.3236
Epoch 2/28
197/197 [==============================] - 602s 3s/step - loss: 4.8518 - accuracy: 0.3585 - val_loss: 4.7908 - val_accuracy: 0.3586
Epoch 3/28
197/197 [==============================] - 595s 3s/step - loss: 4.2763 - accuracy: 0.4079 - val_loss: 4.2936 - val_accuracy: 0.4180
Epoch 4/28
197/197 [==============================] - 595s 3s/step - loss: 3.8175 - accuracy: 0.4498 - val_loss: 4.0312 - val_accuracy: 0.4500
Epoch 5/28
197/197 [==============================] - 632s 3s/step - loss: 3.4409 - accuracy: 0.4855 - val_loss: 4.1645 - val_accuracy: 0.4233
Epoch 6/28
197/197 [==============================] - 596s 3s/step - loss: 3.1141 - accuracy: 0.5204 - val_loss: 3.7770 - val_accuracy: 0.4789
Epoch 7/28
197/197 [==============================] - 591s 3s/step - loss: 2.8344 - accuracy: 0.5508 - val_loss: 3.7240 - val_accuracy: 0.4836

###Manual testing of the translation model with 20 new sentences


In [45]:
import numpy as np
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = sin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom said that he didn't like my dress.
[start] ටොම් කිව්වා එයා මගේ ඇඳුම් අඳින්න කැමති කියලා [end]
-
He tried getting close to her using every means possible.
[start] ඔහු තම පියාගේ අසනීප කිරීමට උත්සාහ කළේ ඔහු සෑම දෙයක්ම ඉතා එයා වෙනස් විය [end]
-
He is not studying English now.
[start] ඔහු දැන් ඉංග්‍රීසි ඉගෙන ගන්න පුරුදු වෙලා [end]
-
Gold is more precious than any other metal.
[start] ඕනෑම එක් පුද්ගලයා ඩොලර් පහක් ඇත [end]
-
She's unfit for the job.
[start] ඇය රැකියාවක් සොයයි [end]
-
Mother cut the cheese with a knife.
[start] අම්මා කුස්සියේ බිම වැටී හිස තබා ගන්න [end]
-
Here are the documents you asked for.
[start] මෙන්න ඔබ සිතන්නේ ඇයි ඔබ කඳු [end]
-
He'd like to have a coffee after work. "I would too."
[start] මම ඔබේ පවුලේ අය සමඟ වැඩ කිරීමට කැමතියි [end]
-
I'm sure Tom will be surprised.
[start] ටොම් සාදයට සහභාගී වීමට මම බලාපොරොත්තු වෙනවා කියලා [end]
-
Quite a few people were present at the meeting yesterday.
[start] ඊයේ රාත්‍රියේ මේ මොහොතේ වැස්ස පටන් ගත්තා [end]
-
Dinosaurs died out 